In [4]:
# A bot that will apply for jobs
url = "https://www.linkedin.com/jobs/search/?currentJobId=3509101579&keywords=marketing%20manager"

url = "https://careers-peraton.icims.com/jobs/131931/software-engineer-ii/candidate?from=login&eem=MWnpnU8MV3MIeioSn89IA93rGjdGFiVZVpBictjqKHLiDJqX%252FfNbjubZV3kx5PlL&code=b7ed6762f06974c7bebe9d05fa1fe50fa33dfcc0b66bbd72e7df19cb72963012&ga=81a5c8777d484570e5f4030406aff2ae558d7345d3ee572bc4471d1023cec550&accept_gdpr=1"
url = "https://www.tesla.com/careers/search/job/apply/159337?source=LinkedIn"

# url = "https://wd1.myworkdaysite.com/en-US/recruiting/fmr/FidelityCareers/job/Boston-MA/Principle-Software-Engineer---Quant-Developer_2071123/apply/"

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import requests
import os
import time
import datetime
import pickle
import atexit



# define a function to save cookies
def save_cookies(driver):
    print("Saving cookies")
    pickle.dump(driver.get_cookies(), open("cookies.pkl", "wb"))


driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()))
driver.get(url)

# load cookies from a file, if available
try:
    cookies = pickle.load(open("cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
except:
    pass


driver.get(url)
print("Here")


Here


In [17]:
save_cookies(driver)

Saving cookies


In [16]:
# find all elements with the aria-label attribute
elements = driver.find_elements(By.XPATH, '//*[@aria-label]')

# loop through the elements and print the aria-label value
for element in elements:
    print(element.get_attribute('aria-label'))

Toast message
LinkedIn
Search by title, skill, or company
Search by title, skill, or company
Search by title, skill, or company
City, state, or zip code
City, state, or zip code
Cancel
Primary Navigation
search filters
Jobs
Dismiss
Cancel Date posted filter
Apply current filter to show results
Date posted filter. Clicking this button displays all Date posted filter options.
Dismiss
Cancel Experience level filter
Apply current filter to show results
Experience level filter. Clicking this button displays all Experience level filter options.
Dismiss
Add a company
Cancel Company filter
Apply current filter to show results
Company filter. Clicking this button displays all Company filter options.
Dismiss
Cancel Job type filter
Apply current filter to show results
Job type filter. Clicking this button displays all Job type filter options.
Dismiss
Cancel On-site/remote filter
Apply current filter to show results
On-site/remote filter. Clicking this button displays all On-site/remote filter opt

In [6]:
# Get text content from the page


# Clean up the html
from bs4 import BeautifulSoup
import html2text
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Key only body
soup = soup.find('body')

# Export as html
html = soup.prettify("utf-8")

# Convert to markdown
markdown = html2text.html2text(html.decode('utf-8'))


# Write to file
with open('page.md', 'w', encoding='utf-8') as f:
    f.write(markdown)


# open ai tokenizer
from transformers import AutoTokenizer

# count number of tokens in html
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.encode(soup.get_text())

# get number of tokens
print(len(tokenizer.encode(soup.get_text())))

# save the html to file
with open('page.html', 'w', encoding='utf-8') as f:
    f.write(soup.prettify())



Token indices sequence length is longer than the specified maximum sequence length for this model (68042 > 1024). Running this sequence through the model will result in indexing errors


68042


In [ ]:
import bs4
import openai


model = "gpt-3.5-turbo"

message = [{
    "content": prompt,
    'role' : "user"
}]

try:
    response = openai.ChatCompletion.create(
        model=model,
        messages=message,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )
except:
    print("error")
    continue

print (response.choices[0].message.content.split("\n"))

In [67]:
# udpate driver with new html
from bs4 import BeautifulSoup
driver.switch_to.window(driver.window_handles[-1])
# get url
url = driver.current_url
print(url)

htmls = []
forms = driver.find_elements(By.XPATH, '//form')
for form in forms:
    inputs = form.find_elements(By.XPATH, '//input')
    labels = form.find_elements(By.XPATH, '//label')
    html = form.get_attribute('innerHTML')
    # pretiffy html remove extra whitespace
    soup = BeautifulSoup(html, 'html.parser')
    html = soup.prettify("utf-8")
    htmls.append(str(html))


print(htmls[0])
# open ai tokenizer
from transformers import AutoTokenizer

# count number of tokens in html
tokenizer = AutoTokenizer.from_pretrained("gpt2")


# get number of tokens
print(len(tokenizer.encode(htmls[0])))


https://wd1.myworkdaysite.com/en-US/recruiting/fmr/FidelityCareers/job/Boston-MA/Principle-Software-Engineer---Quant-Developer_2071123/apply/


IndexError: list index out of range

In [4]:
print(len(forms))

5


In [29]:
from bs4 import Tag
from bs4 import BeautifulSoup
import random

def unwrapper(tags, keep = ('ul','li')):
    for el in tags:
        # print(el.name)
        if isinstance(el,Tag):
            unwrapper(el, keep = keep) # recurse first, unwrap later
            if el.name not in keep:
                el.append("\n")
                el.unwrap()



def remove_more_than_one_newline(text):
    import re
    return re.sub(r'(\n){1,}', r'\n', text)

driver.switch_to.window(driver.window_handles[-1])
url = driver.current_url
print(url)
texts = []
# for form in forms:
#     soup = BeautifulSoup(form.get_attribute('innerHTML'), 'html.parser')
#     unwrapper(soup, keep = ('input', 'br', 'button'))
#     texts.append(soup)

# on the webpage add a id to all buttons and inputs if they don't have one
# then use the id to find the element and click it

elements = driver.find_elements(By.XPATH, "//button | //input | //textarea | //select")

for element in elements:
    if not element.get_attribute("id"):
        element_id = "id_" + str(random.randint(1, 100000))
        driver.execute_script("arguments[0].setAttribute('value',arguments[1])", element, element_id)

# print(driver.page_source)
# load driver into bs4
soup = BeautifulSoup(driver.page_source, 'html.parser')
# drop all style and script tags
for tag in soup(['style', 'script', 'head', 'title', 'meta', '[document]', 'link']):
    tag.decompose()

# soup = BeautifulSoup(driver.get_attribute('innerHTML'), 'html.parser')
unwrapper(soup, keep = ('input', 'br', 'button', 'textarea', 'select'))
texts.append(soup)

print(remove_more_than_one_newline(str(texts[0])))



https://careers-peraton.icims.com/jobs/131931/software-engineer-ii/candidate?from=login&eem=MWnpnU8MV3MIeioSn89IA93rGjdGFiVZVpBictjqKHLiDJqX%252FfNbjubZV3kx5PlL&code=b7ed6762f06974c7bebe9d05fa1fe50fa33dfcc0b66bbd72e7df19cb72963012&ga=81a5c8777d484570e5f4030406aff2ae558d7345d3ee572bc4471d1023cec550&accept_gdpr=1&mobile=false&width=959&height=500&bga=true&needsRedirect=false&jan1offset=-360&jun1offset=-300

Skip Branding
Skip to content
<!-- standard logo -->
 Company
<!--<h4 class="mega-block-title">Learn about Peraton</h4>-->
LEARN ABOUT PERATON
Learn more About Peraton
Peraton supports some of the nation’s most important missions of consequence.
Mission, Vision, and Values
Our Heritage
Our Leaders
Our Commitment to Veterans
Diversity, Equity, and Inclusion
Advisory Board
Partnerships
Markets
<!--<h4 class="mega-block-title">Markets</h4>-->
MARKETS
 LEARN MORE
No matter the national security domain – whether on this planet or beyond – you’ll find Peraton.
Space
Intelligence
Cyber
Defen

In [8]:
from bs4 import Tag
from bs4 import BeautifulSoup
import random

def unwrapper(tags, keep = ('ul','li')):
    for el in tags:
        # print(el.name)
        if isinstance(el,Tag):
            unwrapper(el, keep = keep) # recurse first, unwrap later
            if el.name not in keep:
                el.append("\n")
                el.unwrap()



def remove_more_than_one_newline(text):
    import re
    return re.sub(r'(\n){1,}', r'\n', text)

driver.switch_to.window(driver.window_handles[-1])
url = driver.current_url
print(url)
texts = []
# for form in forms:
#     soup = BeautifulSoup(form.get_attribute('innerHTML'), 'html.parser')
#     unwrapper(soup, keep = ('input', 'br', 'button'))
#     texts.append(soup)

# on the webpage add a id to all buttons and inputs if they don't have one
# then use the id to find the element and click it

elements = driver.find_elements(By.XPATH, "//button | //input | //textarea | //select")

for element in elements:
    if not element.get_attribute("id"):
        element_id = "id_" + str(random.randint(1, 100000))
        driver.execute_script("arguments[0].setAttribute('value',arguments[1])", element, element_id)

# print(driver.page_source)
# load driver into bs4
soup = BeautifulSoup(driver.page_source, 'html.parser')
# drop all style and script tags
for tag in soup(['style', 'script', 'head', 'title', 'meta', 'link']):
    tag.decompose()

# soup = BeautifulSoup(driver.get_attribute('innerHTML'), 'html.parser')
unwrapper(soup, keep = ('input', 'br', 'button', 'textarea', 'select'))
texts.append(soup)

# for each input, textarea, select, button
elements = soup.find_all(['input', 'textarea', 'select', 'button'])

for i, element in enumerate(elements):
    all_html = str(texts[0])
    start_index = all_html.find(str(element))
    end_index = start_index + len(str(element))

    prev_end_index = all_html.find(str(elements[i - 1])) + len(str(elements[i - 1]))



    print(all_html[start_index - 50:end_index + 50])
    print("--------------------------------")


https://www.tesla.com/careers/search/job/apply/159337?source=LinkedIn




Tesla homepage

Careers
Skip to main content

<button action="primaryMobileNavigationItems" class="tds-site-nav-item tds--highlighted tds--product-name" special="menuNavItem" title="Menu" type="button" value="id_10902">Menu
</button>


<button aria-label="Close Modal" class="tds-mod
--------------------------------
" type="button" value="id_10902">Menu
</button>


<button aria-label="Close Modal" class="tds-modal-close" type="button" value="id_56017">

</button>







Job Application
Production Engineering Ass
--------------------------------
Step 1 of 3
Personal Information
Legal First Name
<input aria-invalid="false" class="tds-form-input-text" id="d1c16e46-4aad-4959-9d0c-3dbbed88f53e" name="personal.firstName" required="" value=""/>

Legal Last Name
<input aria-invalid="false" clas
--------------------------------
irstName" required="" value=""/>

Legal Last Name
<input aria-invalid="false" class="tds-form-in